<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2025<br>
<font color=3C99D size=5>
    Project-Phase2<br>
    Soft Actor Critic<br>
<font color=696880 size=4>
    Ali Najar-Mohmmad Shafizade-Armin Khosravi


Nazanin Yousefi - 401110172

MohammadMahdi NouriBorji - 401170061

MohammadJavad Gharegozloo - 401170134


In this notebook, we are going to get familiar with SAC algorithm. Soft Actor Critic (SAC) is an off-policy algorithm that maximizes a combination of expected return **and** entropy. Higher entropy results in higher exploration, which is an important concept in Reinforcement Learning.

## 📦 Setup and Dependencies

Install PyBullet for Physics based environments.


In [128]:
!pip install -q gymnasium pybullet Box2D ffmpeg-python
!nvidia-smi

Fri Sep  5 20:03:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:03:01.0 Off |                    0 |
| N/A   30C    P0             54W /  250W |   33658MiB /  40960MiB |     14%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [129]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [130]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


Import necessary packages.


In [131]:
!pip install "gym==0.21" "pybullet==3.2.5"

Defaulting to user installation because normal site-packages is not writeable


In [132]:
!pip uninstall -y gym gymnasium
!pip install "gym==0.21" "pybullet==3.2.5"


import gym
print("Gym version:", gym.__version__)
print("Gym location:", gym.__file__)


# env = gym.make("HalfCheetahBulletEnv-v0")
# print(env.action_space, env.observation_space)


Found existing installation: gym 0.21.0
Uninstalling gym-0.21.0:
  Successfully uninstalled gym-0.21.0
Found existing installation: gymnasium 1.2.0
Uninstalling gymnasium-1.2.0:
  Successfully uninstalled gymnasium-1.2.0
Defaulting to user installation because normal site-packages is not writeable
  Using cached gym-0.21.0-py3-none-any.whl
Gym version: 0.21.0
Gym location: /home/tavakoli/.local/lib/python3.10/site-packages/gym/__init__.py


In [133]:
import os
import numpy as np
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.distributions import Normal

import warnings
warnings.filterwarnings('ignore')
from gym.wrappers import RecordVideo
import gym
import pybullet_envs
np.bool8 = np.bool_
# from tqdm.notebook import trange
from tqdm import trange
from IPython.display import Video

## 📈 Utility codes

We will use this utility function to visualize the training progress.


In [134]:
def plot_learning_curve(x, filename, save_plot=True):
    avg_x = [np.mean(x[np.max([0, i - 100]):i]) for i in range(len(x))]
    plt.figure(dpi=200)
    plt.title('Learning Curve')
    plt.plot(range(len(x)), x, label='score', alpha=0.3)
    plt.plot(range(len(avg_x)), avg_x, label='average score')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.legend()
    plt.grid()
    if save_plot:
        plt.savefig(filename + '.png')
    plt.show()

This class implements a **Replay Buffer** to store and sample transitions of the form $(s_t, a_t, r_t, s_{t+1}, d_t)$ to break correlation in updates for stability in mini-batch stochastic gradient descent.


In [135]:
class ReplayBuffer:
    def __init__(self, buffer_size, state_dims, action_dims):
        self.buffer_size = buffer_size
        self.ptr = 0
        self.is_full = False


        # TODO: Initialize buffer arrays to store states, next states, actions, rewards, and done flags
        # pass
        self.states = np.zeros((buffer_size, state_dims), dtype=np.float32)
        self.next_states = np.zeros((buffer_size, state_dims), dtype=np.float32)
        self.actions = np.zeros((buffer_size, action_dims), dtype=np.float32)
        self.rewards = np.zeros(buffer_size, dtype=np.float32)
        self.dones = np.zeros(buffer_size, dtype=np.bool_)

    def store_transition(self, state, action, reward, state_, done):
        # TODO : Store the transition in the replay buffer
        self.states[self.ptr] = state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.next_states[self.ptr] = state_
        self.dones[self.ptr] = done

        self.ptr = (self.ptr + 1) % self.buffer_size
        if self.ptr == 0:
            self.is_full = True
        # pass


    def load_batch(self, batch_size):
        # TODO: Sample a random batch of transitions from the buffer
        max_index = self.buffer_size if self.is_full else self.ptr
        indices = np.random.choice(max_index, batch_size, replace=False)

        states = self.states[indices]
        actions = self.actions[indices]
        rewards = self.rewards[indices]
        next_states = self.next_states[indices]
        dones = self.dones[indices]

        return states, actions, rewards, next_states, dones

## 🧱 Neural Networks

This cell defines three core neural networks used in SAC:

- **Critic Network:**
  Estimates the **Q-value function** $ Q(s, a) $. Two critics are used to mitigate overestimation bias.

- **Value Network:**
  Estimates the **state value function** $ V(s) $, used to train the actor and as a baseline.

- **Actor Network:**
  Outputs the **mean** and **standard deviation** for a Gaussian policy
  $
\pi(a|s) = \mathcal{N}(\mu(s), \sigma(s))
$.


In [136]:
class Actor(nn.Module):
    def __init__(self, alpha, state_dims, action_dims, fc1_dims, fc2_dims, max_action, 
                 reparam_noise_lim, name='Actor', ckpt_dir='tmp'):        
        super(Actor, self).__init__()
        
         # TODO 1: Store initialization parameters
        self.alpha = alpha
        self.state_dims = state_dims
        self.action_dims = action_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.max_action = max_action
        self.reparam_noise_lim = reparam_noise_lim  
        self.name = name
        self.ckpt_dir = ckpt_dir
        self.ckpt_path = os.path.join(ckpt_dir, name + '_sac')

        # TODO 2: Define fully connected layers to transform input state
        self.fc1 = nn.Linear(state_dims, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        
        # TODO 3: Output mean and standard deviation for the Gaussian policy
        self.mu = nn.Linear(fc2_dims, action_dims)
        self.sigma = nn.Linear(fc2_dims, action_dims)
        
        # TODO 4: Define optimizer and move model to the appropriate device
        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)


    def forward(self, state):
        #TODO        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        
        mu = self.mu(x)
        sigma = F.softplus(self.sigma(x))  # Ensure positive std dev
        
        return mu, sigma

    def sample_normal(self, state, reparameterize=True):
        # # TODO 7: Use the actor to compute distribution parameters
        mu, sigma = self.forward(state)
        
        # TODO 8: Create a normal distribution and sample from it
        dist = Normal(mu, sigma.clamp(min=self.reparam_noise_lim))
        
        # Sample actions using reparameterization if requested
        if reparameterize:
            actions = dist.rsample()  # Reparameterized sampling
        else:
            actions = dist.sample()   # Standard sampling
        
        # Calculate log probabilities
        log_probs = dist.log_prob(actions).sum(axis=-1, keepdim=True)
        
        # Apply tanh squashing to bound actions
        actions = T.tanh(actions) * T.tensor(self.max_action).to(self.device)
        
        return actions, log_probs

    def save_checkpoint(self):
        os.makedirs(self.ckpt_dir, exist_ok=True)
        T.save(self.state_dict(), self.ckpt_path)

    def load_checkpoint(self, gpu_to_cpu=False):
        if gpu_to_cpu:
            self.load_state_dict(T.load(self.ckpt_path, map_location=lambda storage, loc: storage))
        else:
            self.load_state_dict(T.load(self.ckpt_path))

In [137]:
class Critic(nn.Module):
    def __init__(self, beta, state_dims, action_dims, fc1_dims, fc2_dims, name='Critic', ckpt_dir='tmp'):
        super(Critic, self).__init__()
        
        # TODO 1: Save input arguments as attributes
        self.beta = beta
        self.state_dims = state_dims
        self.action_dims = action_dims
        self.fc1_dims = fc1_dims 
        self.fc2_dims = fc2_dims
        self.name = name
        self.ckpt_dir = ckpt_dir
        self.ckpt_path = os.path.join(ckpt_dir, name + '_sac')
        
        # TODO 2: Define the first fully connected layer
        # The input is the concatenation of state and action
        self.fc1 = nn.Linear(state_dims + action_dims, fc1_dims)
        
        # TODO 3: Define the second hidden layer and the final Q-value output layer
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.q = nn.Linear(fc2_dims, 1)

        # TODO 4: Define the optimizer. Assign to the correct device
        self.optimizer = optim.Adam(self.parameters(), lr=beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state, action):
        # Concatenate state and action
        x = T.cat([state, action], dim=1)
        
        # Forward pass through network
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        q = self.q(x)
        
        return q

    def save_checkpoint(self):
        os.makedirs(self.ckpt_dir, exist_ok=True)
        T.save(self.state_dict(), self.ckpt_path)

    def load_checkpoint(self, gpu_to_cpu=False):
        if gpu_to_cpu:
            self.load_state_dict(T.load(self.ckpt_path, map_location=lambda storage, loc: storage))
        else:
            self.load_state_dict(T.load(self.ckpt_path))

In [138]:
class Value(nn.Module):
    def __init__(self, beta, state_dims, fc1_dims, fc2_dims, name='Value', ckpt_dir='tmp'):
        super(Value, self).__init__()
        
        # TODO 1: Save arguments as instance variables
        self.beta = beta
        self.state_dims = state_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.name = name
        self.ckpt_dir = ckpt_dir
        self.ckpt_path = os.path.join(ckpt_dir, name + '_sac')
        
        # TODO 2: Define the fully connected layers for value approximation
        self.fc1 = nn.Linear(state_dims, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.v = nn.Linear(fc2_dims, 1)
        
        # TODO 3: Set optimizer and device
        self.optimizer = optim.Adam(self.parameters(), lr=beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        v = self.v(x)
        return v

    def save_checkpoint(self):
        os.makedirs(self.ckpt_dir, exist_ok=True)
        T.save(self.state_dict(), self.ckpt_path)

    def load_checkpoint(self, gpu_to_cpu=False):
        if gpu_to_cpu:
            self.load_state_dict(T.load(self.ckpt_path, map_location=lambda storage, loc: storage))
        else:
            self.load_state_dict(T.load(self.ckpt_path))

## 🤖 Agent Implementation

This class encapsulates the full logic of the SAC agent. In general, the learning process uses entropy-regularized policy gradients $J_\pi = \mathbb{E}_{s_t \sim D, a_t \sim \pi} \left[ \alpha \log(\pi(a_t|s_t)) - Q(s_t, a_t) \right]$ with soft target updates $\theta_{\text{target}} \leftarrow \tau \theta + (1 - \tau)\theta_{\text{target}}$.


In [139]:
class Agent:
    def __init__(self, gamma, alpha, beta, state_dims, action_dims, max_action, fc1_dim, fc2_dim,
                 memory_size, batch_size, tau, update_period, reward_scale, warmup, reparam_noise_lim,
                 name, ckpt_dir='tmp'):
        # TODO 1: Save all hyperparameters and paths
        self.gamma = gamma
        self.alpha = alpha  # Entropy coefficient
        self.beta = beta    # Learning rate
        self.state_dims = state_dims
        self.action_dims = action_dims
        self.max_action = max_action
        self.fc1_dim = fc1_dim
        self.fc2_dim = fc2_dim
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.tau = tau
        self.update_period = update_period
        self.reward_scale = reward_scale
        self.warmup = warmup
        self.reparam_noise_lim = reparam_noise_lim
        self.ckpt_dir = ckpt_dir


        model_name = f'{name}__' \
                     f'gamma_{gamma}__' \
                     f'alpha_{alpha}__' \
                     f'beta_{beta}__' \
                     f'fc1_{fc1_dim}__' \
                     f'fc2_{fc2_dim}__' \
                     f'bs_{batch_size}__' \
                     f'buffer_{memory_size}__' \
                     f'update_period_{update_period}__' \
                     f'tau_{tau}__'

        self.model_name = model_name
        self.learn_iter = 0
        self.full_path = os.path.join(self.ckpt_dir, self.model_name)

        # TODO 2: Initialize the replay buffer
        self.memory = ReplayBuffer(memory_size, state_dims, action_dims)
        
        

        # TODO 3: Initialize Actor and Critic networks
        self.actor = Actor(alpha=alpha, state_dims=state_dims, action_dims=action_dims,
                               max_action=max_action, fc1_dims=fc1_dim, fc2_dims=fc2_dim,
                               reparam_noise_lim=reparam_noise_lim, name=f'{name}_actor', ckpt_dir=ckpt_dir)
        
        self.critic_1 = Critic(beta=beta, state_dims=state_dims, action_dims=action_dims,
                                     fc1_dims=fc1_dim, fc2_dims=fc2_dim,
                                     name=f'{name}_critic_1', ckpt_dir=ckpt_dir)
        
        self.critic_2 = Critic(beta=beta, state_dims=state_dims, action_dims=action_dims,
                                     fc1_dims=fc1_dim, fc2_dims=fc2_dim,
                                     name=f'{name}_critic_2', ckpt_dir=ckpt_dir)

        
        
        # TODO 4: Initialize Value and Target Value networks
        self.value = Value(beta=beta, state_dims=state_dims,
                                fc1_dims=fc1_dim, fc2_dims=fc2_dim,
                                name=f'{name}_value', ckpt_dir=ckpt_dir)
        
        self.target_value = Value(beta=beta, state_dims=state_dims,
                                       fc1_dims=fc1_dim, fc2_dims=fc2_dim,
                                       name=f'{name}_target_value', ckpt_dir=ckpt_dir)

        # TODO 5: Sync the parameters of value and target_value networks initially
        self.update_parameters(tau=1.0)

    def choose_action(self, state, deterministic=False, reparameterize=False):
        
        # TODO 6: Convert state to tensor, move to device, and add batch dimension
        state = T.tensor(state, dtype=T.float32).to(self.actor.device).unsqueeze(0)


        # TODO 7: Choose action from actor
        mu, sigma = self.actor(state)
        if deterministic:
            action = mu
        else:
            dist = Normal(mu, sigma.clamp(min=self.reparam_noise_lim))
            action = dist.rsample() if reparameterize else dist.sample()
        
        action = T.tanh(action) * T.tensor(self.max_action).to(self.actor.device)
        
        return action.cpu().detach().numpy()[0]
        # pass

    def store_transition(self, state, action, reward, state_, done):
        # pass
        self.memory.store_transition(state, action, reward, state_, done)

    def load_batch(self):
        states, actions, rewards, states_, dones = self.memory.load_batch(self.batch_size)
        states = T.tensor(states, dtype=T.float32).to(self.actor.device)
        actions = T.tensor(actions, dtype=T.float32).to(self.actor.device)
        rewards = T.tensor(rewards, dtype=T.float32).to(self.actor.device)
        states_ = T.tensor(states_, dtype=T.float32).to(self.actor.device)
        dones = T.tensor(dones, dtype=T.bool).to(self.actor.device)
        return states, actions, rewards, states_, dones

    def update_parameters(self, tau=None):
        # pass
        if tau is None:
            tau = self.tau
        
        target_value_params = self.target_value.named_parameters()
        value_params = self.value.named_parameters()

        target_value_state_dict = dict(target_value_params)
        value_state_dict = dict(value_params)

        for name in target_value_state_dict:
            target_value_state_dict[name] = tau * value_state_dict[name].clone() + \
                                          (1 - tau) * target_value_state_dict[name].clone()

        self.target_value.load_state_dict(target_value_state_dict)



    def save_model(self):
        print('... saving checkpoint ...')
        self.actor.save_checkpoint()
        self.critic_1.save_checkpoint()
        self.critic_2.save_checkpoint()
        self.value.save_checkpoint()
        self.target_value.save_checkpoint()

    def load_model(self, gpu_to_cpu=False):
        print('... loading checkpoint ...')
        self.actor.load_checkpoint(gpu_to_cpu=gpu_to_cpu)
        self.critic_1.load_checkpoint(gpu_to_cpu=gpu_to_cpu)
        self.critic_2.load_checkpoint(gpu_to_cpu=gpu_to_cpu)
        self.value.load_checkpoint(gpu_to_cpu=gpu_to_cpu)
        self.target_value.load_checkpoint(gpu_to_cpu=gpu_to_cpu)

    def learn(self):

        if self.memory.ptr < self.batch_size:
            return

        states, actions, rewards, states_, dones = self.load_batch()

        # === CRITIC LOSS ===
        # Resample actions from the current policy on the next states
        mu, sigma = self.actor(states_)
        dist = Normal(mu, sigma.clamp(min=self.reparam_noise_lim))
        new_actions_next_state = dist.sample()

        # Sum the log probabilities across the action dimensions
        log_probs_next_state = dist.log_prob(new_actions_next_state).sum(-1, keepdim=True)
        new_actions_next_state = T.tanh(new_actions_next_state)

        # Estimate Q-values for the next states
        q1_target_next = self.critic_1(states_, new_actions_next_state).view(-1)
        q2_target_next = self.critic_2(states_, new_actions_next_state).view(-1)
        q_target_next = T.min(q1_target_next, q2_target_next)

        # Compute the value target for the next states
        # Ensure the shape matches the batch size before indexing
        value_target_next = (q_target_next - self.alpha * log_probs_next_state.squeeze()).view(-1).detach()
        value_target_next[dones] = 0.0
        
        # Compute the Q-target for the critics using the bellman equation
        q_target = self.reward_scale * rewards + self.gamma * value_target_next
        
        # Get Q-values for the states and actions from the replay buffer
        q1_old = self.critic_1(states, actions).view(-1)
        q2_old = self.critic_2(states, actions).view(-1)

        # Compute MSE loss for both critics
        critic_1_loss = 0.5 * F.mse_loss(q1_old, q_target.detach())
        critic_2_loss = 0.5 * F.mse_loss(q2_old, q_target.detach())

        self.critic_1.optimizer.zero_grad()
        critic_1_loss.backward()
        self.critic_1.optimizer.step()

        self.critic_2.optimizer.zero_grad()
        critic_2_loss.backward()
        self.critic_2.optimizer.step()

        # === ACTOR LOSS ===
        # Resample actions with reparameterization trick
        mu, sigma = self.actor(states)
        dist = Normal(mu, sigma.clamp(min=self.reparam_noise_lim))
        new_actions = dist.rsample()

        # Sum the log probabilities across the action dimensions
        log_probs = dist.log_prob(new_actions).sum(-1, keepdim=True)
        new_actions = T.tanh(new_actions)
        
        # Get Q-values for the new actions
        q1_new = self.critic_1(states, new_actions)
        q2_new = self.critic_2(states, new_actions)
        q_new = T.min(q1_new, q2_new)

        # Maximize entropy-regularized Q-value
        actor_loss = (self.alpha * log_probs - q_new).mean()
        self.actor.optimizer.zero_grad()
        actor_loss.backward()
        self.actor.optimizer.step()

        # === TARGET NETWORK UPDATE ===
        if self.learn_iter % self.update_period == 0:
            self.update_parameters()

        self.learn_iter += 1


## ⚙️ Training Configuration

Set up your training parameters. `HalfCheetahBulletEnv-v0` is a continuous control task where the agent must learn to run using articulated legs.


In [ ]:
# Configuration parameters


env_name = 'HalfCheetahBulletEnv-v0'
dir = 'tmp'
n_games = 1000


# gamma = 0.995
# alpha = 5e-5  # Entropy coefficient
# beta = 5e-5   # Learning rate
# fc1_dim = 256
# fc2_dim = 256
# memory_size = 1000000  # Replay buffer size
# batch_size = 256       # Batch size for training
# tau = 0.005
# update_period = 2
# reward_scale = 1.0
# warmup = 20000  # Warmup steps before learning
# reparam_noise_lim = 1e-6
# record_video = True

gamma = 0.99
alpha = 0.1  
beta = 1e-4 
fc1_dim = 256
fc2_dim = 256
memory_size = 1_000_000
batch_size = 256
tau = 0.005
update_period = 2
reward_scale = 1.0
warmup = 10000  
reparam_noise_lim = 1e-6
n_games = 2000

## 🚀 Training Loop

For each episode, interact with the environment to collect transitions, then update the SAC networks and save the best model.

After training, a learning curve is plotted to visualize convergence and performance stability.


In [141]:
!brew install ffmpeg

/bin/bash: line 1: brew: command not found


In [142]:
env = gym.make(env_name)
dir_path = os.path.join(dir, env_name)
os.makedirs(dir_path, exist_ok=True)

if record_video:
    env = RecordVideo(env, video_folder=os.path.join(dir_path, 'videos'),
                      episode_trigger=lambda ep: ep == n_games - 1)
    
    

# agent = None

agent = Agent(
    gamma=gamma,
    alpha=alpha,
    beta=beta,
    state_dims=env.observation_space.shape[0],
    action_dims=env.action_space.shape[0],
    max_action=float(env.action_space.high[0]),
    fc1_dim=fc1_dim,
    fc2_dim=fc2_dim,
    memory_size=memory_size,
    batch_size=batch_size,
    tau=tau,
    update_period=update_period,
    reward_scale=reward_scale,
    warmup=warmup,
    reparam_noise_lim=reparam_noise_lim,
    name='sac_halfcheetah',
    ckpt_dir=dir
)



# TODO: Initialize performance tracking variables
scores = []
avg_scores = []
best_score = float('-inf')


for game in trange(n_games):
    # TODO: Reset environment and initialize variables at the start of each episode
    state = env.reset()
    if isinstance(state, tuple):  # Handle gym API returning (state, info)
        state = state[0]
    done = False
    truncated = False
    score = 0

    # TODO: Interact with the environment until the episode is done
    while not done and not truncated:
        action = agent.choose_action(state)
        step_result = env.step(action)
        
        # Handle both old and new gym API formats
        if len(step_result) == 5:
            next_state, reward, done, truncated, info = step_result
        else:
            next_state, reward, done, info = step_result
            truncated = False
            
        # Consider episode done if terminated or truncated
        done = done or truncated
        agent.store_transition(state, action, reward, next_state, done)
        agent.learn()
        state = next_state
        score += reward


    # TODO: Track score and average score for plotting and saving
    scores.append(score)
    avg_score = np.mean(scores[-100:]) if len(scores) >= 100 else np.mean(scores)
    avg_scores.append(avg_score)
    if avg_score > best_score:
        best_score = avg_score

    print(f'| Game: {game:6.0f} | Score: {score:10.2f} | Best score: {best_score:10.2f} | '
          f'Avg score {avg_score:10.2f} | Learning iter: {agent.learn_iter:10.0f} |')


    # TODO: Save the model if the current avg score is better than the best so far
    if avg_score >= best_score:
        agent.save_model()

env.close()

plot_learning_curve(scores, agent.full_path)

  0%|          | 0/2000 [00:00<?, ?it/s]

argv[0]=
argv[0]=


  0%|          | 1/2000 [00:08<4:46:09,  8.59s/it]

| Game:      0 | Score:   -1525.38 | Best score:   -1525.38 | Avg score   -1525.38 | Learning iter:        745 |
... saving checkpoint ...


  0%|          | 2/2000 [00:19<5:29:21,  9.89s/it]

...

| Game:    511 | Score:   -1730.86 | Best score:   -1525.38 | Avg score   -1643.41 | Learning iter:     511745 |


 26%|██▌       | 513/2000 [2:12:12<6:44:15, 16.31s/it]

| Game:    512 | Score:   -1708.37 | Best score:   -1525.38 | Avg score   -1643.62 | Learning iter:     512745 |


 26%|██▌       | 513/2000 [2:12:25<6:23:50, 15.49s/it]


ValueError: Expected parameter loc (Tensor of shape (1, 6)) of distribution Normal(loc: torch.Size([1, 6]), scale: torch.Size([1, 6])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

## 🎥 Visualize Agent Behavior

This is the last episode recorded video of the trained agent interacting with the environment in training process.


In [ ]:
Video(f"/content/tmp/HalfCheetahBulletEnv-v0/videos/rl-video-episode-{n_games-1}.mp4", embed=True, width=600)